1️⃣ Setup e Configuração

In [22]:
# Imports necessários
from pinecone import Pinecone, ServerlessSpec
from langchain_huggingface import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
import time
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_core.documents import Document

# Carregar variáveis de ambiente
load_dotenv()

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


/Users/iagocampista/anaconda3/envs/travelAgent/lib/python3.11/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [23]:
# Configuração das credenciais
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT', 'us-east-1')
INDEX_NAME = "assistente-viagem"

# Inicializar cliente Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Verificar índices existentes
print("📋 Índices existentes:")
print()
print("=" * 20)

for index in pc.list_indexes().names():
    print(f" [] {index}")
    print("=" * 20)

# cria índice se não existir
if INDEX_NAME not in [i.name for i in pc.list_indexes()]:
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,  # depende do modelo de embeddings
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(INDEX_NAME)

📋 Índices existentes:

 [] aula-ia-01
 [] assistente-viagem


In [29]:
# 1. Importe a lista de dados do arquivo separado
from base_conhecimento import DADOS_DE_CONHECIMENTO

# 2. Transformar os dados em Documentos LangChain
documentos = []
for item in DADOS_DE_CONHECIMENTO:
    doc = Document(
        page_content=item["conteudo"],
        metadata={
            "cidade": item["cidade"],
            "categoria": item["categoria"],
            "subtopico": item["subtopico"]
        }
    )
    documentos.append(doc)

# 3. Usar um TextSplitter para caso o chunck pre-estabelecido tenha ficado muito longo
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=50
)

chunks_finais = text_splitter.split_documents(documentos)

# 4. Verifique o resultado
print(f"Número de documentos originais: {len(documentos)}")
print(f"Número total de chunks gerados: {len(chunks_finais)}")
print("\nExemplo do primeiro chunk gerado:")
print(chunks_finais[2])

print("\nExemplo de um chunk de Paris:")
print(chunks_finais[-1])

Número de documentos originais: 20
Número total de chunks gerados: 20

Exemplo do primeiro chunk gerado:
page_content='A Praia de Copacabana é ideal para caminhadas no calçadão e visitas a quiosques. Durante o réveillon, recebe milhões de turistas.' metadata={'cidade': 'Rio de Janeiro', 'categoria': 'Atrações', 'subtopico': 'Praia de Copacabana'}

Exemplo de um chunk de Paris:
page_content='Há muitos carteiristas em locais turísticos e no metrô; mantenha bolsas e mochilas sempre próximas. Evite carregar muito dinheiro em espécie; prefira cartões. Para deslocamentos noturnos em áreas afastadas, utilize táxis ou aplicativos de transporte.' metadata={'cidade': 'Paris', 'categoria': 'Segurança', 'subtopico': 'Precauções Gerais'}


Uplpad para o PineCone

In [30]:
print("Iniciando o processo de upload para o Pinecone...")

# 2. Inicializar o modelo de Embeddings do Hugging Face
print("Carregando o modelo de embeddings do Hugging Face...")
# O modelo será baixado e armazenado em cache na primeira vez que for executado
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {'device': 'cpu'} # Forçar o uso da CPU
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
print("Modelo de embeddings carregado com sucesso.")

# 3. Inicializar a conexão com o Pinecone
print("Conectando ao Pinecone...")
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index_name = "assistente-viagem" # O nome do seu índice com a dimensão correta (384)
print(f"Conexão estabelecida. Usando o índice: {index_name}")

# 4. Vetorizar e Fazer o Upload dos Chunks
# O LangChain faz todo o trabalho pesado aqui:
# - Pega cada chunk da lista 'chunks_finais'
# - Usa o modelo do Hugging Face para criar um embedding (vetor)
# - Envia o vetor e os metadados para o índice do Pinecone
print(f"Iniciando o upload de {len(chunks_finais)} chunks. Isso pode levar alguns minutos...")

docsearch = PineconeVectorStore.from_documents(
    documents=chunks_finais,
    embedding=embeddings,
    index_name=index_name
)

print("\n--- SUCESSO! ---")
print("Todos os chunks foram vetorizados e enviados para o seu índice no Pinecone.")
print("Você já pode verificar o 'Vector Count' no seu dashboard do Pinecone.")

Iniciando o processo de upload para o Pinecone...
Carregando o modelo de embeddings do Hugging Face...
Modelo de embeddings carregado com sucesso.
Conectando ao Pinecone...
Conexão estabelecida. Usando o índice: assistente-viagem
Iniciando o upload de 20 chunks. Isso pode levar alguns minutos...

--- SUCESSO! ---
Todos os chunks foram vetorizados e enviados para o seu índice no Pinecone.
Você já pode verificar o 'Vector Count' no seu dashboard do Pinecone.


In [ ]:
#Para caso seja necessario apagar os dados do pinecone (debugging)
#index.delete(delete_all=True)

{}

In [32]:
# Teste de busca simples
query = "pontos turísticos famosos"
query_embedding = embeddings.embed_query(query)

# Buscar documentos similares
results = index.query(
    vector=query_embedding,
    top_k=3,
    include_metadata=True
)

print(f"🔍 **TESTE DE BUSCA:** '{query}'")
print("\n📋 **RESULTADOS:**")
for i, match in enumerate(results['matches'], 1):
    score = match['score']
    text = match['metadata']['text']
    cidade = match['metadata']['cidade']
    
    print(f"\n{i}. **{cidade.upper()}** (Score: {score:.3f})")
    print(f"   {text}")

🔍 **TESTE DE BUSCA:** 'pontos turísticos famosos'

📋 **RESULTADOS:**

1. **RIO DE JANEIRO** (Score: 0.628)
   A Praia de Copacabana é ideal para caminhadas no calçadão e visitas a quiosques. Durante o réveillon, recebe milhões de turistas.

2. **PARIS** (Score: 0.532)
   Os passeios de barco pelo Rio Sena permitem conhecer os principais monumentos de Paris de uma perspectiva diferente.

3. **RIO DE JANEIRO** (Score: 0.528)
   Durante grandes eventos como carnaval e réveillon, é importante reservar hospedagem e passagens com antecedência.
